In [ ]:
import sqlite3
import pandas as pd

In [ ]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [ ]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [ ]:
data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [ ]:
data.to_sql("boston", con, index=False, if_exists="replace")

In [ ]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [ ]:
data.shape

(506, 14)

# 1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [ ]:
pd.read_sql(
    """
    SELECT 'CRIM' as 'feature', COUNT(CRIM) as 'empty_count' FROM boston
    WHERE CRIM is Null
    UNION
    SELECT 'ZN', COUNT(ZN) FROM boston
    WHERE ZN is Null
    UNION
    SELECT 'INDUS', COUNT(INDUS) FROM boston
    WHERE INDUS is Null
    UNION
    SELECT 'CHAS', COUNT(CHAS) FROM boston
    WHERE CHAS is Null
    UNION
    SELECT 'NOX', COUNT(NOX) FROM boston
    WHERE NOX is Null
    """,
    con,
)

,feature,empty_count
0,CHAS,0
1,CRIM,0
2,INDUS,0
3,NOX,0
4,ZN,0


# 2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [ ]:
pd.read_sql(
    """
    SELECT 'CRIM' as 'feature', COUNT(DISTINCT CRIM) as 'unique_count' FROM boston
    UNION
    SELECT 'ZN', COUNT(DISTINCT ZN) FROM boston
    UNION
    SELECT 'INDUS', COUNT(DISTINCT INDUS) FROM boston
    UNION
    SELECT 'CHAS', COUNT(DISTINCT CHAS) FROM boston
    UNION
    SELECT 'NOX', COUNT(DISTINCT NOX) FROM boston
    """,
    con,
)

,feature,unique_count
0,CHAS,2
1,CRIM,504
2,INDUS,76
3,NOX,81
4,ZN,26


# 3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [ ]:
pd.read_sql(
    """
    WITH new_table AS(
    SELECT * FROM (
        (SELECT 'CRIM' as 'feature', AVG(CRIM) as "median"
        FROM (SELECT CRIM
              FROM boston
              ORDER BY CRIM
              LIMIT 2
              OFFSET (SELECT (COUNT(*) - 1) / 2
                      FROM boston))),
        (SELECT MIN(CRIM) as "min" FROM boston))
    UNION
    SELECT * FROM (
        (SELECT 'ZN' as 'feature', AVG(ZN) as "median"
        FROM (SELECT ZN
              FROM boston
              ORDER BY ZN
              LIMIT 2
              OFFSET (SELECT (COUNT(*) - 1) / 2
                      FROM boston))),
        (SELECT MIN(ZN) as "min" FROM boston))
    UNION
    SELECT * FROM (
        (SELECT 'INDUS' as 'feature', AVG(INDUS) as "median"
        FROM (SELECT INDUS
              FROM boston
              ORDER BY INDUS
              LIMIT 2
              OFFSET (SELECT (COUNT(*) - 1) / 2
                      FROM boston))),
        (SELECT MIN(INDUS) as "min" FROM boston))
    UNION
    SELECT * FROM (
        (SELECT 'CHAS' as 'feature', AVG(CHAS) as "median"
        FROM (SELECT CHAS
              FROM boston
              ORDER BY CHAS
              LIMIT 2
              OFFSET (SELECT (COUNT(*) - 1) / 2
                      FROM boston))),
        (SELECT MIN(CHAS) as "min" FROM boston))
    UNION
    SELECT * FROM (
        (SELECT 'NOX' as 'feature', AVG(NOX) as "median"
        FROM (SELECT NOX
              FROM boston
              ORDER BY NOX
              LIMIT 2
              OFFSET (SELECT (COUNT(*) - 1) / 2
                      FROM boston))),
        (SELECT MIN(NOX) as "min" FROM boston))
    )
    
    SELECT * FROM new_table
    WHERE median = min
    """,
    con,
)
    
    

,feature,median,min
0,CHAS,0.0,0.0
1,ZN,0.0,0.0


Вывод: Большая часть значений в колонках CHAS и ZN имеют минимальное значение.

# 4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [ ]:
pd.read_sql(
    """
    
    WITH 
    chip AS (
    SELECT '25' as build_counts, AVG(RM) as avg_chip
    FROM (SELECT * FROM boston ORDER BY MEDV LIMIT 25)
    UNION
    SELECT '50', AVG(RM)
    FROM (SELECT * FROM boston ORDER BY MEDV LIMIT 50)
    UNION
    SELECT '100', AVG(RM)
    FROM (SELECT * FROM boston ORDER BY MEDV LIMIT 100)
    UNION
    SELECT '200', AVG(RM)
    FROM (SELECT * FROM boston ORDER BY MEDV LIMIT 200)
    UNION
    SELECT '300', AVG(RM)
    FROM (SELECT * FROM boston ORDER BY MEDV LIMIT 300)
    ),
    expensive AS (
    SELECT AVG(RM) as avg_exp
    FROM boston
    WHERE MEDV = (SELECT MAX(MEDV) FROM boston))
    
    SELECT cast(build_counts as INTEGER) as build_counts,
    avg_chip, avg_exp, avg_exp-avg_chip AS Diff
    FROM chip, expensive
    ORDER BY build_counts
    """,
    con,
)

,build_counts,avg_chip,avg_exp,Diff
0,25,5.747840,7.484,1.736160
1,50,5.753240,7.484,1.730760
2,100,5.887120,7.484,1.596880
3,200,5.911705,7.484,1.572295
4,300,5.972227,7.484,1.511773


Вывод: В общем наблюдается увеличение стоимости квартир при увеличении среднего числа комнат.

# 5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [ ]:
pd.read_sql(
    """
    SELECT * FROM (SELECT LSTAT, MEDV,  rank() OVER (ORDER BY LSTAT) as rank
                    FROM boston)
    WHERE MEDV = (SELECT MAX(MEDV) FROM boston)

    """,
    con,
)

,LSTAT,MEDV,rank
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


Вывод: в дорогих домах низкий процент населения с низким статусом

# 6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [ ]:
pd.read_sql(
    """
    SELECT DISTINCT (CASE WHEN CHAS=1 THEN 'граничит' ELSE 'не граничит' END) as river, 
    AVG(MEDV) OVER (PARTITION BY CHAS) as avg_price
    FROM boston

    """,
    con,
)

,river,avg_price
0,не граничит,22.093843
1,граничит,28.440000


# 7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [ ]:
pd.read_sql(
    """

    WITH table1 AS (
    SELECT
        CASE WHEN AVG(CRIM) < (SELECT AVG(CRIM) FROM boston WHERE CHAS = 1) THEN 'CRIM' ELSE 'NO' END flag
    FROM boston
    UNION
    SELECT
        CASE WHEN AVG(ZN) < (SELECT AVG(ZN) FROM boston WHERE CHAS = 1) THEN 'ZN' ELSE 'NO' END flag
    FROM boston
    UNION
    SELECT
        CASE WHEN AVG(INDUS) < (SELECT AVG(INDUS) FROM boston WHERE CHAS = 1) THEN 'INDUS' ELSE 'NO' END flag
    FROM boston
    UNION
    SELECT
        CASE WHEN AVG(NOX) < (SELECT AVG(NOX) FROM boston WHERE CHAS = 1) THEN 'NOX' ELSE 'NO' END flag
    FROM boston
    )
    SELECT *
    FROM table1
    WHERE flag != 'NO'
 
    """,
    con,
)

,flag
0,INDUS
1,NOX


Вывод: рядом с рекой в среднем больше объектов неторгового бизнеса чем во всем городе, также рядом с рекой увеличена концентрация оксида озота

# 8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [ ]:
pd.read_sql(
    """
    SELECT percentile * 10 AS percentile, AVG(INDUS) AS perc_INDUS, AVG(NOX) AS perc_NOX
    FROM (SELECT NTILE(10) OVER (ORDER BY INDUS, NOX) AS percentile, INDUS, NOX
        FROM boston)
    GROUP BY percentile
    """,
    con,
)

,percentile,perc_INDUS,perc_NOX
0,10,2.016078,0.438378
1,20,3.645294,0.494129
2,30,5.246275,0.448235
3,40,6.471765,0.470839
4,50,8.256078,0.531510
5,60,10.749216,0.515176
6,70,17.245800,0.564920
7,80,18.100000,0.684880
8,90,18.307200,0.714620
9,100,21.938800,0.692860


Вывод: с увеличением доли акров неторгового бизнеса в районе увеличивается концентрация оксидов азота.